In [2]:
import pandas as pd

In [3]:
df= pd.read_csv('..\db\Content.csv')

C:\Users\sande\AppData\Local\Temp\ipykernel_7300\1989823227.py:1: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('..\db\Content.csv')


In [4]:
df.head()

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
0,1,1,Nutrient,4,29,Kiwi,Actinidia chinensis PLANCHON [Actinidiaceae],Fruit,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,1955.00,raw,0
1,2,1,Nutrient,6,53,Onion,Allium cepa L. [Liliaceae],Bulb,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,1853.95,raw,0
2,3,1,Nutrient,6,53,Onion,Allium cepa L. [Liliaceae],Leaf,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,4150.00,raw,0
3,4,1,Nutrient,9,55,Chives,Allium schoenoprasum L. [Liliaceae],Leaf,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,3900.00,raw,0
4,5,1,Nutrient,11,70,Cashew,Anacardium occidentale L. [Anacardiaceae],Fruit,FAT,FAT,...,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,2500.00,other,0


## Analysis of DB

In [9]:
df.shape

(5145532, 26)

In [13]:
df['orig_food_scientific_name'].value_counts()

Camellia sinensis (L.) KUNTZE [Theaceae]    4242
Capsicum annuum L. [Solanaceae]             1840
Daucus carota L. [Apiaceae]                  748
Zea mays L. [Poaceae]                        672
Apium graveolens L. [Apiaceae]               644
                                            ... 
Curcuma domestica                              1
Anthriscus cerefolium                          1
Coffea robusta                                 1
Bos taurus                                     1
Arctium lappa L.                               1
Name: orig_food_scientific_name, Length: 678, dtype: int64

In [11]:
df['orig_food_scientific_name'].describe()

count                                        46097
unique                                         678
top       Camellia sinensis (L.) KUNTZE [Theaceae]
freq                                          4242
Name: orig_food_scientific_name, dtype: object

In [12]:
df['orig_food_scientific_name'].isna().sum()

5099435

Analysis summary:

Out of the 5145532 rows, 5099435 are Nan ie have missing scientific names. Out of the 46097 fields without Nan scientific names, many are duplicates and only 678 unique scientific names are present.
In addition, the scientific names aren't in standard genus species notation. For eg. Apium graveolens L. [Apiaceae]. Need to do some string manipulation to fix this.

## Extacting relevant data from db

In [22]:
# dropping na values in scientific name columnn
df_drop = df.dropna(subset=['orig_food_scientific_name'])
df_drop.shape

(46097, 26)

In [23]:
df_drop.drop_duplicates(subset=['orig_food_scientific_name'], keep='first', inplace=True)

C:\Users\sande\AppData\Local\Temp\ipykernel_7300\2033690832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop.drop_duplicates(subset=['orig_food_scientific_name'], keep='first', inplace=True)


In [24]:
df_drop.shape

(678, 26)

In [26]:
df_drop['orig_food_common_name'].isna().sum()

438

In [30]:
df_drop =df_drop.dropna(subset=['orig_food_common_name'])
df_drop.shape

(240, 26)

In [31]:
df_drop.columns

Index(['id', 'source_id', 'source_type', 'food_id', 'orig_food_id',
       'orig_food_common_name', 'orig_food_scientific_name', 'orig_food_part',
       'orig_source_id', 'orig_source_name', 'orig_content', 'orig_min',
       'orig_max', 'orig_unit', 'orig_citation', 'citation', 'citation_type',
       'creator_id', 'updater_id', 'created_at', 'updated_at', 'orig_method',
       'orig_unit_expression', 'standard_content', 'preparation_type',
       'export'],
      dtype='object')

In [33]:
final = df_drop[['food_id','orig_food_common_name','orig_food_scientific_name']]

Extraction Summary

Out of the 678 unique scientific names, only 240 had a matching common name. So we drop all na values of common names. Finally we chose the relevant columns in the df named final

In [45]:
final.iloc[1,:]

food_id                                               6
orig_food_common_name                             Onion
orig_food_scientific_name    Allium cepa L. [Liliaceae]
scientific_name                             Allium cepa
other                                    L. [Liliaceae]
Name: 1, dtype: object

The scientific name is of a weird format. Need to standardize this

In [37]:
def std_name(name):
    word_list = name.split()
    sci_name = word_list[0]+' '+word_list[1]
    other = ' '.join(word_list[2:])
    return sci_name,other

In [42]:
final[['scientific_name', 'other']] = final['orig_food_scientific_name'].apply(lambda x: pd.Series(std_name(x)))

C:\Users\sande\AppData\Local\Temp\ipykernel_7300\1100743060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final[['scientific_name', 'other']] = final['orig_food_scientific_name'].apply(lambda x: pd.Series(std_name(x)))
C:\Users\sande\AppData\Local\Temp\ipykernel_7300\1100743060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final[['scientific_name', 'other']] = final['orig_food_scientific_name'].apply(lambda x: pd.Series(std_name(x)))


In [43]:
final.head()

,food_id,orig_food_common_name,orig_food_scientific_name,scientific_name,other
0,4,Kiwi,Actinidia chinensis PLANCHON [Actinidiaceae],Actinidia chinensis,PLANCHON [Actinidiaceae]
1,6,Onion,Allium cepa L. [Liliaceae],Allium cepa,L. [Liliaceae]
3,9,Chives,Allium schoenoprasum L. [Liliaceae],Allium schoenoprasum,L. [Liliaceae]
4,11,Cashew,Anacardium occidentale L. [Anacardiaceae],Anacardium occidentale,L. [Anacardiaceae]
7,12,Pineapple,Ananas comosus (L.) MERR. [Bromeliaceae],Ananas comosus,(L.) MERR. [Bromeliaceae]


In [44]:
final.shape

(240, 5)

In [47]:
final.to_csv('../db/common_name.csv', index=True)